# 0. Problem statement

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn, and identify the main indicators of churn.

In this competition, your goal is *to build a machine learning model that is able to predict churning customers based on the features provided for their usage.*

**Customer behaviour during churn:**

Customers usually do not decide to switch to another competitor instantly, but rather over a
period of time (this is especially applicable to high-value customers). In churn prediction, we
assume that there are three phases of customer lifecycle :

1. <u>The ‘good’ phase:</u> In this phase, the customer is happy with the service and behaves as usual.

2. <u>The ‘action’ phase:</u> The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. It is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)

3. <u>The ‘churn’ phase:</u> In this phase, the customer is said to have churned. In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month (September) is the ‘churn’ phase.

**Evaluation**
<br>**Goal**
<br>It is your job to predict if a customer will churn, given the ~170 columns containing customer behavior, usage patterns, payment patterns, and other features that might be relevant. Your target variable is "churn_probability"

**Metric**
<br>Submissions are evaluated on Classification Accuracy between the value of the predicted value and the actual value of churn for each of the customers.

**Accuracy score formula**

<br>The public leaderboard is going to rank your submission against other users while the competition is active, however, once the competition is ended, the final ranks will be calculated on the private leaderboard.

**Submission file format**
<br>The file should contain a header and have the following format (CSV):

**id,churn_probability**
<br>70005,0.0
<br>70006,1.0
<br>70007,0.0
<br>etc.


# Steps or Summary :
1. Import Necessary Libraries
2. Load the Data and Understanding the Data
    - <i>Segmentation of Columns</i>
    - <i>Missing Value Check</i>
    - <i>Outlier Treatment</i>
3. Exploratory Data Analysis:
    - <i>Univariate Analysis (One Variable at a time)</i>
    - <i>Bivariate Analysis (Two Variable at a time)</i>
    - <i>Multivariate Analysis (More than two Variables at a time)</i>
4. Data Preprocessing and Data Preparation Steps
    - <i>Data Preparation Steps : Dummy Variable Creation (One Hot Encoding)</i>
    - <i>Train-Test Split : Splitting the Data into Training and Testing Sets</i>
    - <i>Feature Scaling : Standard scaling </i>
5. Building the initial model ( Model 0 )
6. Feature Selection : Using RFE and Manual Selection methods
7. Check and Build the models using selected features
8. Residual Analysis of the train data
9. Making Predictions using the final model
10. Steps for Further Model Refinement and Optimization
11. Model Evaluation

# 1. Import Necessary Libraries

In [124]:
#Data Structures
import pandas as pd
import numpy as np
import re
import os
import time

### For installing missingno library, type this command in terminal
#pip install missingno

import missingno as msno

#Statsmodel
import statsmodels as sm

#Sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report

#Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

#Others
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 2. Load the Data and Understanding the Data

In [125]:
#COMMENT THIS SECTION INCASE RUNNING THIS NOTEBOOK LOCALLY

#Checking the kaggle paths for the uploaded datasets
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [126]:
#INCASE RUNNING THIS LOCALLY, PASS THE RELATIVE PATH OF THE CSV FILES BELOW
#(e.g. if files are in same folder as notebook, simple write "train.csv" as path)

#churn_data = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c-63/train.csv")
#churn_data_unseen = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c-63/test.csv")
#sample = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c-63/sample.csv")
#data_dict = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c-63/data_dictionary.csv")

In [ ]:
### Load and read data
churn_data = pd.read_csv("train.csv")
churn_data_unseen = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")
data_dict = pd.read_csv("data_dictionary.csv")

print(churn_data.shape)
print(churn_data_unseen.shape)
print(sample.shape)
print(data_dict.shape)

In [ ]:
churn_data.head()

In [ ]:
churn_data_unseen.head()

### Info of datasets

In [ ]:
churn_data.info(verbose=True)

In [ ]:
churn_data_unseen.info(verbose=True)

### Basic summary statistics

In [ ]:
churn_data.describe(include="all")

In [ ]:
churn_data_unseen.describe(include="all")

## Some custom defined functions

In [ ]:
# Define a function to rename columns based on regex patterns
def rename_columns(col_name):
    # Patterns and replacements
    patterns = {
        r'_6$': '_jun',
        r'_7$': '_jul',
        r'_8$': '_aug'
    }
    
    for pattern, replacement in patterns.items():
        if re.search(pattern, col_name):
            return re.sub(pattern, replacement, col_name)
    return col_name

In [ ]:
# Define a function to rename columns based on regex patterns
def rename_columns2(col_name):
    # Define replacements for prefixes
    if col_name.startswith('jun_'):
        return re.sub(r'^jun_', '', col_name) + '_jun'
    elif col_name.startswith('jul_'):
        return re.sub(r'^jul_', '', col_name) + '_jul'
    elif col_name.startswith('aug_'):
        return re.sub(r'^aug_', '', col_name) + '_aug'
    else:
        return col_name

In [ ]:
# Function to check if a column has the same value in all rows
def is_column_constant_custom(df, cols , val=1):
    return df[cols].apply(lambda x: x.dropna().nunique() <= val and x.notna().all())

# Print the number of columns in the training set where all values are the same
constant_cols_train = churn_data.columns[is_column_constant_custom(churn_data, churn_data.columns)]
print("Seen Set: Number of columns where all values are the same:", len(constant_cols_train))
print("Columns with constant values in Seen Set:", constant_cols_train.tolist())
# Separator line
print("="*120)
# Print the number of columns in the test set where all values are the same
constant_cols_test = churn_data_unseen.columns[is_column_constant_custom(churn_data_unseen, churn_data_unseen.columns)]
print("Unseen Set: Number of columns where all values are the same:", len(constant_cols_test))
print("Columns with constant values in Unseen Set:", constant_cols_test.tolist())

In [ ]:
def findCustomColumnTypes(df, search_type='all_search', target_column=None, include_all=False,  keyword=None):
    """
    Finds and returns column names in a DataFrame based on various types and patterns.
    
    Parameters:
    - df: pandas DataFrame
      The DataFrame in which to search for column names.
    
    - search_type: string, optional, default='all_search'
      Specifies the type of search to perform. Can be one of:
        - 'all_search': Search across all specified types.
        - 'date_text': Search for columns with 'date' in their names.
        - 'object_type': Search for columns with object data types.
        - 'datetime': Search for columns with datetime data types.
        - 'numerical': Search for columns with numerical data types.
        - 'categorical': Search for columns with categorical features.
      
    - include_all: boolean, optional, default=False
      If True, includes all column names in the result under the key 'all_columns'.
    
    - keyword: string, optional
      A custom string pattern to match in column names. If provided, and if 
      `search_type` matches this key, the function will return columns that match the pattern.

    - target_column: string, optional
      A column name to be excluded from the results, if present. This will be removed from 
      lists of columns for each search type where applicable.

    Returns:
    - result: dict
      A dictionary containing lists of column names based on the search type.
      The keys of the dictionary will be:
        - 'all_columns': List of all columns (if `include_all=True`).
        - 'date_text_columns': Columns containing 'date' in their names.
        - 'object_type_columns': Columns with object data types.
        - 'datetime_columns': Columns with datetime data types.
        - 'numerical_columns': Columns with numerical data types.
        - 'categorical_columns': Columns identified as categorical.
        - '{key}_columns': Columns matching the custom `key` pattern (if provided).
    """
    
    # List all column names in the DataFrame
    all_columns = sorted(list(df.columns))
    result = {}
    
    # Include all columns if specified
    if include_all:
        result['all_columns'] = all_columns

    # Search for columns containing 'date' in their names
    if search_type == 'all_search' or search_type == 'date_text' or search_type == 'categorical':
        pattern = re.compile(r'date', re.IGNORECASE)
        date_text_columns = sorted([col for col in all_columns if pattern.search(col)])
        # Remove target_column from date_text_columns if it exists
        if target_column in date_text_columns:
            date_text_columns.remove(target_column)
        result['date_text_columns'] = date_text_columns
    
    # Search for columns with object data types
    if search_type == 'all_search' or search_type == 'object_type' or search_type == 'categorical':
        object_type_columns = sorted(list(df.select_dtypes(include=['object']).columns))
        # Remove target_column from object_type_columns if it exists
        if target_column in object_type_columns:
            object_type_columns.remove(target_column)
        result['object_type_columns'] = object_type_columns
    
    # Search for columns with datetime data types
    if search_type == 'all_search' or search_type == 'datetime' or search_type == 'categorical':
        datetime_columns = sorted(list(df.select_dtypes(include=['datetime64[ns]', '<M8[ns]']).columns))
        # Remove target_column from datetime_columns if it exists
        if target_column in datetime_columns:
            datetime_columns.remove(target_column)
        result['datetime_columns'] = datetime_columns
    
    # Search for numerical columns
    if search_type == 'all_search' or search_type == 'numerical' or search_type == 'categorical':
        numerical_columns = sorted(list(df.select_dtypes(include=['int64', 'float64']).columns))
        # Remove target_column from numerical_columns if it exists
        if target_column in numerical_columns:
            numerical_columns.remove(target_column)
        result['numerical_columns'] = numerical_columns
    
    # Search for categorical columns based on unique values
    if search_type == 'all_search' or search_type == 'categorical':
        constant_cols_train = df.columns[is_column_constant_custom(df, df.columns, 7)]
        categorical_columns = sorted(list(set(constant_cols_train) - set(numerical_columns) - set(object_type_columns) - set(datetime_columns)))
        # Remove target_column from categorical_columns if it exists
        if target_column in categorical_columns:
            categorical_columns.remove(target_column)
        result['categorical_columns'] = categorical_columns
    
    # Search for custom key in column names
    if keyword:
        if search_type != 'all_search' or search_type == "key":
            pattern = re.compile(keyword, re.IGNORECASE)
            filtered_columns = sorted([col for col in all_columns if pattern.search(col)])
            # Remove target_column from filtered_columns if it exists
            if target_column in filtered_columns:
                filtered_columns.remove(target_column)
            result[f'{keyword}_columns'] = filtered_columns
    
    return result

In [ ]:
def getMissingValues(data, missing_cutoff_value, dataset_type="Seen", vars_type="Numerical"):
    """
    Identify and print columns with missing values above a specified cutoff percentage.

    Parameters:
    data (DataFrame): The dataset to analyze.
    missing_cutoff_value (float): The cutoff percentage for missing values to filter columns.
    dataset_type (str): Type of dataset, e.g., "Seen" or "Unseen". Default is "Seen".
    vars_type (str): Type of variables, e.g., "Numerical" or "Categorical". Default is "Numerical".

    Returns:
    list: List of column names with missing values above the cutoff percentage.
    """
    # Calculate the percentage of missing values for each column
    missing_data = round(100 * data.isnull().mean())
    # Filter columns where missing values exceed the cutoff percentage and sort them in descending order
    missing_data_above_cutoff = missing_data[missing_data > missing_cutoff_value].sort_values(ascending=False)
    # Print the columns with missing values above the cutoff percentage
    print(f"----- {vars_type} variables of {dataset_type} dataset having missing values above cutoff value ({missing_cutoff_value}%) : -----\n{missing_data_above_cutoff}")
    # Find columns with missing values above the cutoff percentage
    missing_data_above_cutoff_cols = list(missing_data_above_cutoff.index)
    # Print the total number of columns with missing values above the cutoff and list of these columns
    print(f"Total number of {vars_type} variables of {dataset_type} dataset having missing values above cutoff value ({missing_cutoff_value}%) : {len(missing_data_above_cutoff_cols)}")
    #print(f"-----  {vars_type} variables of {dataset_type} dataset having missing values : -----\n{missing_data_above_cutoff_cols}")
    # Return the list of columns with missing values above the cutoff
    return missing_data_above_cutoff_cols

In [ ]:
def simpleImputeMissingValues(data, data_unseen, cols, dec="constant", var=0):
    """
    Impute missing values in both training and test datasets.

    Parameters:
    data (DataFrame): The training dataset.
    data_unseen (DataFrame): The unseen (test) dataset.
    cols (list): List of column names to impute.
    dec (str): Imputation strategy. Options are 'constant', 'mean', 'median', etc.
    var (int, float, optional): Value to replace missing values with if `dec` is 'constant'. Default is 0.

    Returns:
    None: The function modifies the input DataFrames in place and prints the shapes of the datasets.
    """
    # Set up the imputer based on the chosen strategy
    if dec == "constant":
        imputer = SimpleImputer(strategy=dec, fill_value=var)
    else:
        imputer = SimpleImputer(strategy=dec)
    # Apply the imputer to the training dataset
    data[cols] = imputer.fit_transform(data[cols])
    # Apply the same imputer to the unseen (test) dataset
    data_unseen[cols] = imputer.transform(data_unseen[cols])
    # Print the shape of the DataFrame after imputation for the training set
    print("Seen Set Shape :- ", data.shape)
    # Separator line for clarity
    print("=" * 120)
    # Print the shape of the DataFrame after imputation for the test set
    print("Unseen Set Shape :- ", data_unseen.shape)

In [ ]:
def impute_rows(row, col1, col2, col3, val):
    """
    Impute missing values in a row based on specified conditions.

    Parameters:
    row (pd.Series): A row from the DataFrame.
    col1 (str): The name of the primary column.
    col2 (str): The name of the first secondary column.
    col3 (str): The name of the second secondary column.
    val: The value to impute if the columns are missing.

    Returns:
    pd.Series: The updated row with imputed values.
    """
    # Print the row ID
    print(f"Processing row with index {row.name}")
    
    # Check and handle col1
    if pd.isnull(row[col1]):
        print(f"Row {row.name}: {col1} is null. Imputing with value {val}.")
        row[col1] = val
    else:
        print(f"Row {row.name}: {col1} is already not null, value - {row[col1]}")
        
    # Check and handle col2
    if pd.notnull(row[col1]) and pd.isnull(row[col2]):
        print(f"Row {row.name}: {col2} is null. Imputing with {col1} value {row[col1]}.")
        row[col2] = row[col1]
    elif pd.isnull(row[col1]) and pd.isnull(row[col2]):
        print(f"Row {row.name}: Both {col1} and {col2} are null. Imputing {col2} with value {val}.")
        row[col2] = val
    else:
        print(f"Row {row.name}: {col2} is already not null, value - {row[col2]}")
        
    # Check and handle col3
    if pd.notnull(row[col1]) and pd.isnull(row[col2]) and pd.isnull(row[col3]):
        print(f"Row {row.name}: {col3} is null. Imputing with {col1} value {row[col1]}.")
        row[col3] = row[col1]
    elif pd.notnull(row[col2]) and pd.isnull(row[col3]):
        print(f"Row {row.name}: {col3} is null. Imputing with {col2} value {row[col2]}.")
        row[col3] = row[col2]
    elif pd.isnull(row[col1]) and pd.isnull(row[col2]) and pd.isnull(row[col3]):
        print(f"Row {row.name}: All three columns are null. Imputing {col3} with value {val}.")
        row[col3] = val
    else:
        print(f"Row {row.name}: {col3} is already not null, value - {row[col3]}")
        
    return row

In [ ]:
def remove_outliers(df, numeric_cols, lower_quantile=0.10, upper_quantile=0.90, iqr_multiplier=1.5):
    """
    Removes outliers from the specified numeric columns in the DataFrame based on the interquartile range method.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - numeric_cols (list): List of column names in df to remove outliers from.
    - lower_quantile (float): The lower percentile threshold (default is 0.10).
    - upper_quantile (float): The upper percentile threshold (default is 0.90).
    - iqr_multiplier (float): The multiplier for the interquartile range to define outlier boundaries (default is 1.5).

    Returns:
    - pd.DataFrame: A new DataFrame with outliers removed.
    """
    filtered_df = df.copy()
    for col in numeric_cols:
        # Calculate the quantiles
        q1 = filtered_df[col].quantile(lower_quantile)
        q3 = filtered_df[col].quantile(upper_quantile)
        iqr = q3 - q1
        # Calculate the lower and upper bounds for outliers
        range_low = q1 - iqr_multiplier * iqr
        range_high = q3 + iqr_multiplier * iqr
        ### # Print the quantile boundaries for debugging
        ### print(f"Column: {col}")
        ### print(f"  10th Percentile (Q1): {q1}")
        ### print(f"  90th Percentile (Q3): {q3}")
        ### print(f"  IQR: {iqr}")
        ### print(f"  Lower Bound: {range_low}")
        ### print(f"  Upper Bound: {range_high}")
        # Filter the DataFrame
        filtered_df = filtered_df[(filtered_df[col] > range_low) & (filtered_df[col] < range_high)]
        # Print the shape after filtering
        print(f"Shape after filtering column {col}: {filtered_df.shape}")
    return filtered_df

### Let's replace columns suffixes _6, _7, and _8 with jun, jul, and aug, respectively, using regular expressions

In [ ]:
# Apply the renaming function to all column names
churn_data.rename(columns=lambda x: rename_columns(x), inplace=True)
churn_data_unseen.rename(columns=lambda x: rename_columns(x), inplace=True)

### Looking at the data we can see that there are a few columns which do not have the proper naming convention where the month is appended in the end. Lets fix them.

In [ ]:
# Apply the renaming function 2 to all column names
churn_data.rename(columns=lambda x: rename_columns2(x), inplace=True)
churn_data_unseen.rename(columns=lambda x: rename_columns2(x), inplace=True)

### Let's check info of datasets to check all columns

In [ ]:
# List all column names in the DataFrame
all_columns = list(churn_data.columns)
print("------- Columns for Seen Set ------- \n",all_columns)
# Separator line
print("="*120) 
print("------- Columns for Unseen Set ------- \n",list(churn_data_unseen.columns))

### Check duplicate records

In [ ]:
churn_data[churn_data.duplicated()]

In [ ]:
churn_data_unseen[churn_data_unseen.duplicated()]

### To check all rows in some particular columns are null. If all rows are null then we'll remove those rows or records

In [ ]:
# Print the number of rows in the training set where all columns are missing
print("Seen Set: Rows with all columns missing:", churn_data[churn_data.isnull().all(axis=1)].shape[0])
# Separator line
print("="*120) 
# Print the number of rows in the test set where all columns are missing
print("Unseen Set: Rows with all columns missing:", churn_data_unseen[churn_data_unseen.isnull().all(axis=1)].shape[0])

### Checking for Columns with Constant Values (Considering NaN as unique value) 

### We'll drop "circle_id","last_date_of_month_jun" columns from Seen and Unseen datasets.

In [ ]:
churn_data.drop(["circle_id","last_date_of_month_jun"],axis=1,inplace=True)
churn_data_unseen.drop(["circle_id","last_date_of_month_jun"],axis=1,inplace=True)

### Segmentation of Columns

In [ ]:
target_col='churn_probability'

In [ ]:
churn_data_columns=findCustomColumnTypes(df=churn_data, search_type='all_search', target_column=target_col, include_all=False,  keyword=None)
churn_data_unseen_columns=findCustomColumnTypes(df=churn_data_unseen, search_type='all_search', target_column=target_col, include_all=False,  keyword=None)

In [ ]:
### Identify the object columns
print("------- Object Columns for Seen Set ------- \n",churn_data_columns["object_type_columns"])
# Separator line
print("="*120)
print("------- Object Columns for Unseen Set ------- \n",churn_data_unseen_columns["object_type_columns"])
# Separator line
print("="*120)
### Identify the numerical columns
print("------- Numerical Columns for Seen Set ------- \n",churn_data_columns["numerical_columns"])
# Separator line
print("="*120) 
print("------- Numerical Columns for Unseen Set ------- \n",churn_data_unseen_columns["numerical_columns"])

### Missing value check

### Check missing values for non-object columns or variables

In [ ]:
## Lets check the columns with missing values greater than 40%
cut_off=40
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["numerical_columns"]],cut_off,"Seen","Numerical")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["numerical_columns"]],cut_off,"Unseen","Numerical")

### Many of these columns provide insights into customer behavior, spending, and engagement, all of which are crucial for understanding and predicting churn. Dropping them could result in the loss of valuable information that might be predictive of churn.

### Now we cannot simply remove these columns since they are important and mainly suggest that there has not been significant revenue generated from these customers. So we'll have to impute the missing values. 

In [ ]:
churn_data[missing_values_cols_seen].describe(include="all")

### Based on the above summary statistics of 27 columns, it appears that imputing the missing values by setting them to 0 is a sensible approach, rather than replacing the missing values with the mean or average.  A missing value in these columns might actually indicate that there was no usage (i.e., 0 minutes). Imputing these missing values with the mean would imply some level of usage when, in reality, there might have been none.

In [ ]:
# missing_values_cols contains the columns to impute
imputeCols = missing_values_cols_seen
# Set up the imputer to replace missing values with 0
simpleImputeMissingValues(churn_data, churn_data_unseen, imputeCols, dec="constant", var=0)

In [ ]:
## Lets check the columns with missing values greater than 10%
cut_off=10
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["numerical_columns"]],cut_off,"Seen","Numerical")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["numerical_columns"]],cut_off,"Unseen","Numerical")

In [ ]:
## Lets check the columns with missing values 
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["numerical_columns"]],cut_off,"Seen","Numerical")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["numerical_columns"]],cut_off,"Unseen","Numerical")

### Imputing with 0 aligns with the business logic that no usage (missing data) means 0 minutes or times. Here, setting missing values to 0 in this scenario better reflects the underlying reality of the data. So, let's impute with zero

In [ ]:
# missing_values_cols contains the columns to impute
imputeCols = missing_values_cols_seen
# Set up the imputer to replace missing values with 0
simpleImputeMissingValues(churn_data, churn_data_unseen, imputeCols, dec="constant", var=0)

In [ ]:
## Lets check the columns with missing values 
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["numerical_columns"]],cut_off,"Seen","Numerical")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["numerical_columns"]],cut_off,"Unseen","Numerical")

### Check missing values for object columns

In [ ]:
## Lets check the object columns with missing values greater than cut off
cut_off=40
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["object_type_columns"]],cut_off,"Seen","Object")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["object_type_columns"]],cut_off,"Unseen","Object")

###   More than 70% data is blank in these 'date_of_last_rech_data_jun', 'date_of_last_rech_data_jul', 'date_of_last_rech_data_aug' columns , so for now we will drop these 3 columns

In [ ]:
### Create a copy or backup
churn_data_copy1=churn_data.copy()
churn_data_unseen_copy1=churn_data_unseen.copy()

In [ ]:
# missing_values_cols contains the columns to impute
imputeCols = missing_values_cols_seen
## dropping these columns
churn_data.drop(imputeCols,axis=1,inplace=True)
## dropping these columns
churn_data_unseen.drop(imputeCols,axis=1,inplace=True)
# Print the shape of the DataFrame after imputation for the training set
print("Seen Set Shape :- ", churn_data.shape)
# Separator line for clarity
print("=" * 120)
# Print the shape of the DataFrame after imputation for the test set
print("Unseen Set Shape :- ", churn_data_unseen.shape)
churn_data_columns=findCustomColumnTypes(churn_data,"all_search",target_col)
churn_data_unseen_columns=findCustomColumnTypes(churn_data_unseen,"all_search",target_col)

In [ ]:
## Lets check the object columns with missing values
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["object_type_columns"]],cut_off,"Seen","Object")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["object_type_columns"]],cut_off,"Unseen","Object")

In [ ]:
churn_data[missing_values_cols_seen].describe()

### Since "last_date_of_month_jul" and "last_date_of_month_aug" represent specific reference dates, we will impute the missing values with these respective dates.

In [ ]:
# Columns that need imputation
imputeCols =['last_date_of_month_jul', 'last_date_of_month_aug']
## dropping these columns
churn_data.drop(imputeCols,axis=1,inplace=True)
## dropping these columns
churn_data_unseen.drop(imputeCols,axis=1,inplace=True)
# Print the shape of the DataFrame after imputation for the training set
print("Seen Set Shape :- ", churn_data.shape)
# Separator line for clarity
print("=" * 120)
# Print the shape of the DataFrame after imputation for the test set
print("Unseen Set Shape :- ", churn_data_unseen.shape)
churn_data_columns=findCustomColumnTypes(churn_data,"all_search",target_col)
churn_data_unseen_columns=findCustomColumnTypes(churn_data_unseen,"all_search",target_col)

In [ ]:
## Lets check the object columns with missing values
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data[churn_data_columns["object_type_columns"]],cut_off,"Seen","Object")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen[churn_data_columns["object_type_columns"]],cut_off,"Unseen","Object")

In [ ]:
# Columns that need imputation
imputeCols = missing_values_cols_seen
# Set up the SimpleImputer to replace missing values with the most frequent value
simpleImputeMissingValues(churn_data, churn_data_unseen, imputeCols, dec="most_frequent", var=False)

In [ ]:
churn_data[missing_values_cols_seen].describe()

In [ ]:
## Lets check the object columns with missing values 
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data,cut_off,"Seen","all types")
# Separator line
print("="*120) 
missing_values_cols_unseen=getMissingValues(churn_data_unseen,cut_off,"Unseen","all types")

### There are no missing values now

In [ ]:
## removing columns which have only 0 as values in them
zeros=list(churn_data.columns[(churn_data == 0).all()])
churn_data[zeros].head()

### Let's drop the 9 columns where all values are identical (i.e., 0) for all rows

In [ ]:
## dropping these columns
churn_data.drop(zeros,axis=1,inplace=True)
## dropping these columns
churn_data_unseen.drop(zeros,axis=1,inplace=True)
# Print the shape of the DataFrame after imputation for the training set
print("Seen Set Shape :- ", churn_data.shape)
# Separator line for clarity
print("=" * 120)
# Print the shape of the DataFrame after imputation for the test set
print("Unseen Set Shape :- ", churn_data_unseen.shape)
churn_data_columns=findCustomColumnTypes(churn_data,"all_search",target_col)
churn_data_unseen_columns=findCustomColumnTypes(churn_data_unseen,"all_search",target_col)

### Convert the data types of date columns to datetime

In [ ]:
## converting the datatypes of dates to datetime
for col in churn_data_columns["date_text_columns"]:
    churn_data[col] = pd.to_datetime(churn_data[col], format='%m/%d/%Y')
    
## converting the datatypes of dates to datetime
for col in churn_data_unseen_columns["date_text_columns"]:
    churn_data_unseen[col] = pd.to_datetime(churn_data_unseen[col], format='%m/%d/%Y')

In [ ]:
churn_data[churn_data_columns["date_text_columns"]].head()

### Identify different of Business Metrics Variables sets

In [55]:
findCustomColumnTypes(churn_data,"key",target_col,False,r"amt")

{'amt_columns': ['av_rech_amt_data_aug',
  'av_rech_amt_data_jul',
  'av_rech_amt_data_jun',
  'last_day_rch_amt_aug',
  'last_day_rch_amt_jul',
  'last_day_rch_amt_jun',
  'max_rech_amt_aug',
  'max_rech_amt_jul',
  'max_rech_amt_jun',
  'total_rech_amt_aug',
  'total_rech_amt_jul',
  'total_rech_amt_jun']}

In [61]:
findCustomColumnTypes(churn_data, search_type="key", target_column=target_col, include_all=False, key="(rch|rech)")

{'(rch|rech)_columns': []}

In [ ]:
# Define a regular expression pattern to match columns with 'date'
pattern = re.compile(r'amt', re.IGNORECASE)
# Use list comprehension to filter columns based on the pattern
amount_columns = [col for col in list(churn_data.columns) if pattern.search(col)]

# Define a regular expression pattern to match columns with 'date'
pattern = re.compile(r'(rch|rech)', re.IGNORECASE)
# Use list comprehension to filter columns based on the pattern
recharge_columns = [col for col in list(churn_data.columns) if pattern.search(col)]

In [ ]:
print(list(set(amount_columns) | set(recharge_columns) | set(data_columns)))

### High Value Customer
#### In this section, we will identify high-value customers based on business objectives.

We will determine high-value customers based on their monthly recharge amount spending. Total Monthly Recharge Amount (total_month_rech_amt) is calculated by summing up the recorded recharge transactions (total_rech_amt_) with any additional or updated recharge data (total_rech_amt_data) to get a complete view of a customer's monthly spending. To do this, we will calculate the total monthly recharge amount using the following formula:
` total_month_rech_amt = total_rech_amt + ( total_rech_data * av_rech_amt_data ) `

We will also introduce a new metric that aggregates the recharge amounts across all months.


In [ ]:
churn_data[["total_rech_amt_jun","total_rech_data_jun","av_rech_amt_data_jun"]].head()

In [ ]:
# List of month suffixes
months = ['jun', 'jul', 'aug']

### Find Total Data Recharge Amount 

In [ ]:
# Calculate total recharge amount data for each month in seen set
churn_data['total_month_rech_amt_jun'] = churn_data['total_rech_amt_jun'] + churn_data['total_rech_data_jun'] * churn_data['av_rech_amt_data_jun']
churn_data['total_month_rech_amt_jul'] = churn_data['total_rech_amt_jul'] + churn_data['total_rech_data_jul'] * churn_data['av_rech_amt_data_jul']
churn_data['total_month_rech_amt_aug'] = churn_data['total_rech_amt_aug'] + churn_data['total_rech_data_aug'] * churn_data['av_rech_amt_data_aug']

# Calculate total recharge amount data for each month in unseen set
churn_data_unseen['total_month_rech_amt_jun'] = churn_data_unseen['total_rech_amt_jun'] + churn_data_unseen['total_rech_data_jun'] * churn_data_unseen['av_rech_amt_data_jun']
churn_data_unseen['total_month_rech_amt_jul'] = churn_data_unseen['total_rech_amt_jul'] + churn_data_unseen['total_rech_data_jul'] * churn_data_unseen['av_rech_amt_data_jul']
churn_data_unseen['total_month_rech_amt_aug'] = churn_data_unseen['total_rech_amt_aug'] + churn_data_unseen['total_rech_data_aug'] * churn_data_unseen['av_rech_amt_data_aug']

In [ ]:
churn_data.head()

### Filtering High-Value Prepaid Customers Based on Business Objectives
In this section, we'll focus on identifying high-value prepaid customers by considering usage-based churn. We'll analyze the recharge amounts for June and July and select the top 30% of customers as high-value based on their spending.

### Calculate average spending in first two months. The first two months are the ‘good’ phase
<u>The ‘good’ phase:</u> In this phase, the customer is happy with the service and behaves as usual.

In [ ]:
# Calculate the average recharge amount for the months of  June, and July
# by taking the mean of the total recharge amounts for each month.

# Adding a new column to churn_data DataFrame with the average recharge amount for June, and July
churn_data["AVG_amt_jun_jul"] = churn_data[["total_month_rech_amt_jun", "total_month_rech_amt_jul"]].mean(axis=1)
churn_data[["total_month_rech_amt_jun", "total_month_rech_amt_jul", "total_month_rech_amt_aug","AVG_amt_jun_jul"]].head()

In [ ]:
# Get summary statistics for the 'AVG_amt_jun_jul_aug' column with custom percentiles
churn_data[["total_month_rech_amt_jun", "total_month_rech_amt_jul", "total_month_rech_amt_aug","AVG_amt_jun_jul"]].describe(percentiles=[0, 0.25, 0.50, 0.70, 0.75, 0.80, 0.85,0.90,0.95, 1.0])

In [ ]:
# finding the cutoff value of good phase which is the 70th percentile of the good phase average recharge amounts
churn_data_high=churn_data[ churn_data["AVG_amt_jun_jul"] > churn_data["AVG_amt_jun_jul"].quantile(0.7) ]
## resetting the index
churn_data_high.reset_index(inplace=True,drop=True)
# Drop the "AVG_amt_jun_jul" column from the filtered DataFrame
churn_data_high = churn_data_high.drop("AVG_amt_jun_jul", axis=1)
# Print the shape of the DataFrame for high-value customers
print("Shape of the dataset for high-value customers:", churn_data_high.shape)

### Derive New Features :

### Find Total for each month and Mean incoming and outgoing information  across Different Call Types (Local , STD , ISD and Roaming)

In [ ]:
# Calculate the total local minutes of usage for each month
churn_data_high['total_loc_mou_jun'] = churn_data_high['loc_og_mou_jun'] + churn_data_high['loc_ic_mou_jun']
churn_data_high['total_loc_mou_jul'] = churn_data_high['loc_og_mou_jul'] + churn_data_high['loc_ic_mou_jul']
churn_data_high['total_loc_mou_aug'] = churn_data_high['loc_og_mou_aug'] + churn_data_high['loc_ic_mou_aug']

# Calculate the mean of total local minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_loc_mou'] = round(churn_data_high[['total_loc_mou_jun', 'total_loc_mou_jul', 'total_loc_mou_aug']].mean(axis=1),2)

# Calculate the total STD minutes of usage for each month
churn_data_high['total_std_mou_jun'] = churn_data_high['std_og_mou_jun'] + churn_data_high['std_ic_mou_jun']
churn_data_high['total_std_mou_jul'] = churn_data_high['std_og_mou_jul'] + churn_data_high['std_ic_mou_jul']
churn_data_high['total_std_mou_aug'] = churn_data_high['std_og_mou_aug'] + churn_data_high['std_ic_mou_aug']

# Calculate the mean of total STD minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_std_mou'] = round(churn_data_high[['total_std_mou_jun', 'total_std_mou_jul', 'total_std_mou_aug']].mean(axis=1),2)

# Calculate the total ISD minutes of usage for each month
churn_data_high['total_isd_mou_jun'] = churn_data_high['isd_og_mou_jun'] + churn_data_high['isd_ic_mou_jun']
churn_data_high['total_isd_mou_jul'] = churn_data_high['isd_og_mou_jul'] + churn_data_high['isd_ic_mou_jul']
churn_data_high['total_isd_mou_aug'] = churn_data_high['isd_og_mou_aug'] + churn_data_high['isd_ic_mou_aug']

# Calculate the mean of total ISD minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_isd_mou'] = round(churn_data_high[['total_isd_mou_jun', 'total_isd_mou_jul', 'total_isd_mou_aug']].mean(axis=1),2)

# Calculate the total Roaming minutes of usage for each month
churn_data_high['total_roam_mou_jun'] = churn_data_high['roam_og_mou_jun'] + churn_data_high['roam_ic_mou_jun']
churn_data_high['total_roam_mou_jul'] = churn_data_high['roam_og_mou_jul'] + churn_data_high['roam_ic_mou_jul']
churn_data_high['total_roam_mou_aug'] = churn_data_high['roam_og_mou_aug'] + churn_data_high['roam_ic_mou_aug']

# Calculate the mean of total Roaming minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_roam_mou'] = round(churn_data_high[['total_roam_mou_jun', 'total_roam_mou_jul', 'total_roam_mou_aug']].mean(axis=1),2)


# Repeat the same calculations for churn_data_unseen DataFrame
# Calculate the total local minutes of usage for each month
churn_data_unseen['total_loc_mou_jun'] = churn_data_unseen['loc_og_mou_jun'] + churn_data_unseen['loc_ic_mou_jun']
churn_data_unseen['total_loc_mou_jul'] = churn_data_unseen['loc_og_mou_jul'] + churn_data_unseen['loc_ic_mou_jul']
churn_data_unseen['total_loc_mou_aug'] = churn_data_unseen['loc_og_mou_aug'] + churn_data_unseen['loc_ic_mou_aug']

# Calculate the mean of total local minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_loc_mou'] = round(churn_data_unseen[['total_loc_mou_jun', 'total_loc_mou_jul', 'total_loc_mou_aug']].mean(axis=1),2)

# Calculate the total STD minutes of usage for each month
churn_data_unseen['total_std_mou_jun'] = churn_data_unseen['std_og_mou_jun'] + churn_data_unseen['std_ic_mou_jun']
churn_data_unseen['total_std_mou_jul'] = churn_data_unseen['std_og_mou_jul'] + churn_data_unseen['std_ic_mou_jul']
churn_data_unseen['total_std_mou_aug'] = churn_data_unseen['std_og_mou_aug'] + churn_data_unseen['std_ic_mou_aug']

# Calculate the mean of total STD minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_std_mou'] = round(churn_data_unseen[['total_std_mou_jun', 'total_std_mou_jul', 'total_std_mou_aug']].mean(axis=1),2)

# Calculate the total ISD minutes of usage for each month
churn_data_unseen['total_isd_mou_jun'] = churn_data_unseen['isd_og_mou_jun'] + churn_data_unseen['isd_ic_mou_jun']
churn_data_unseen['total_isd_mou_jul'] = churn_data_unseen['isd_og_mou_jul'] + churn_data_unseen['isd_ic_mou_jul']
churn_data_unseen['total_isd_mou_aug'] = churn_data_unseen['isd_og_mou_aug'] + churn_data_unseen['isd_ic_mou_aug']

# Calculate the mean of total ISD minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_isd_mou'] = round(churn_data_unseen[['total_isd_mou_jun', 'total_isd_mou_jul', 'total_isd_mou_aug']].mean(axis=1),2)

# Calculate the total Roaming minutes of usage for each month
churn_data_unseen['total_roam_mou_jun'] = churn_data_unseen['roam_og_mou_jun'] + churn_data_unseen['roam_ic_mou_jun']
churn_data_unseen['total_roam_mou_jul'] = churn_data_unseen['roam_og_mou_jul'] + churn_data_unseen['roam_ic_mou_jul']
churn_data_unseen['total_roam_mou_aug'] = churn_data_unseen['roam_og_mou_aug'] + churn_data_unseen['roam_ic_mou_aug']

# Calculate the mean of total Roaming minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_roam_mou'] = round(churn_data_unseen[['total_roam_mou_jun', 'total_roam_mou_jul', 'total_roam_mou_aug']].mean(axis=1),2)

### Metrics for Total ARPU for each month and Mean Value from 2G and 3G Services

In [ ]:
# Calculate the total ARPU from 2G and 3G services for each month for churn_data_high
churn_data_high['total_arpu_2g_3g_jun'] = churn_data_high['arpu_2g_jun'] + churn_data_high['arpu_3g_jun']
churn_data_high['total_arpu_2g_3g_jul'] = churn_data_high['arpu_2g_jul'] + churn_data_high['arpu_3g_jul']
churn_data_high['total_arpu_2g_3g_aug'] = churn_data_high['arpu_2g_aug'] + churn_data_high['arpu_3g_aug']

# Calculate the mean of total ARPU from 2G and 3G services across June, July, and August for churn_data_high
churn_data_high['mean_total_arpu_2g_3g'] = round(churn_data_high[['total_arpu_2g_3g_jun', 'total_arpu_2g_3g_jul', 'total_arpu_2g_3g_aug']].mean(axis=1), 2)

# Calculate the total ARPU from 2G and 3G services for each month for churn_data_unseen
churn_data_unseen['total_arpu_2g_3g_jun'] = churn_data_unseen['arpu_2g_jun'] + churn_data_unseen['arpu_3g_jun']
churn_data_unseen['total_arpu_2g_3g_jul'] = churn_data_unseen['arpu_2g_jul'] + churn_data_unseen['arpu_3g_jul']
churn_data_unseen['total_arpu_2g_3g_aug'] = churn_data_unseen['arpu_2g_aug'] + churn_data_unseen['arpu_3g_aug']

# Calculate the mean of total ARPU from 2G and 3G services across June, July, and August for churn_data_unseen
churn_data_unseen['mean_total_arpu_2g_3g'] = round(churn_data_unseen[['total_arpu_2g_3g_jun', 'total_arpu_2g_3g_jul', 'total_arpu_2g_3g_aug']].mean(axis=1), 2)

### Metrics for Total Data Volume for each month and Mean Volume from 2G and 3G Services

In [ ]:
# Calculate the total data volume from 2G and 3G services for each month for churn_data_high
churn_data_high['total_vol_2g_3g_mb_jun'] = churn_data_high['vol_2g_mb_jun'] + churn_data_high['vol_3g_mb_jun']
churn_data_high['total_vol_2g_3g_mb_jul'] = churn_data_high['vol_2g_mb_jul'] + churn_data_high['vol_3g_mb_jul']
churn_data_high['total_vol_2g_3g_mb_aug'] = churn_data_high['vol_2g_mb_aug'] + churn_data_high['vol_3g_mb_aug']

# Calculate the mean of total data volume from 2G and 3G services across June, July, and August for churn_data_high
churn_data_high['mean_total_vol_2g_3g_mb'] = round(churn_data_high[['total_vol_2g_3g_mb_jun', 'total_vol_2g_3g_mb_jul', 'total_vol_2g_3g_mb_aug']].mean(axis=1), 2)

# Calculate the total data volume from 2G and 3G services for each month for churn_data_unseen
churn_data_unseen['total_vol_2g_3g_mb_jun'] = churn_data_unseen['vol_2g_mb_jun'] + churn_data_unseen['vol_3g_mb_jun']
churn_data_unseen['total_vol_2g_3g_mb_jul'] = churn_data_unseen['vol_2g_mb_jul'] + churn_data_unseen['vol_3g_mb_jul']
churn_data_unseen['total_vol_2g_3g_mb_aug'] = churn_data_unseen['vol_2g_mb_aug'] + churn_data_unseen['vol_3g_mb_aug']

# Calculate the mean of total data volume from 2G and 3G services across June, July, and August for churn_data_unseen
churn_data_unseen['mean_total_vol_2g_3g_mb'] = round(churn_data_unseen[['total_vol_2g_3g_mb_jun', 'total_vol_2g_3g_mb_jul', 'total_vol_2g_3g_mb_aug']].mean(axis=1), 2)

### Metrics for Total On-net and Off-net Minutes of Usage for each Month and Mean 

In [ ]:
# Calculate the total on-net and off-net minutes of usage for each month for churn_data_high
churn_data_high['total_onnet_offnet_mou_jun'] = churn_data_high['onnet_mou_jun'] + churn_data_high['offnet_mou_jun']
churn_data_high['total_onnet_offnet_mou_jul'] = churn_data_high['onnet_mou_jul'] + churn_data_high['offnet_mou_jul']
churn_data_high['total_onnet_offnet_mou_aug'] = churn_data_high['onnet_mou_aug'] + churn_data_high['offnet_mou_aug']

# Calculate the mean of total on-net and off-net minutes of usage across June, July, and August for churn_data_high
churn_data_high['mean_total_onnet_offnet_mou'] = round(churn_data_high[['total_onnet_offnet_mou_jun', 'total_onnet_offnet_mou_jul', 'total_onnet_offnet_mou_aug']].mean(axis=1), 2)

# Calculate the total on-net and off-net minutes of usage for each month for churn_data_unseen
churn_data_unseen['total_onnet_offnet_mou_jun'] = churn_data_unseen['onnet_mou_jun'] + churn_data_unseen['offnet_mou_jun']
churn_data_unseen['total_onnet_offnet_mou_jul'] = churn_data_unseen['onnet_mou_jul'] + churn_data_unseen['offnet_mou_jul']
churn_data_unseen['total_onnet_offnet_mou_aug'] = churn_data_unseen['onnet_mou_aug'] + churn_data_unseen['offnet_mou_aug']

# Calculate the mean of total on-net and off-net minutes of usage across June, July, and August for churn_data_unseen
churn_data_unseen['mean_total_onnet_offnet_mou'] = round(churn_data_unseen[['total_onnet_offnet_mou_jun', 'total_onnet_offnet_mou_jul', 'total_onnet_offnet_mou_aug']].mean(axis=1), 2)


### Identify "t2t" columns

In [ ]:
# Define regex pattern to match columns containing "t2t"
pattern = r't2t'
# Create a list to hold columns containing "t2t"
t2t_cols = sorted([col for col in churn_data_high.columns if re.search(pattern, col, re.IGNORECASE)])
# Print identified columns
print('Columns with "t2t":', t2t_cols)

In [ ]:
# Calculate the total local T2T minutes of usage for each month
churn_data_high['total_loc_t2t_mou_jun'] = churn_data_high['loc_og_t2t_mou_jun'] + churn_data_high['loc_ic_t2t_mou_jun']
churn_data_high['total_loc_t2t_mou_jul'] = churn_data_high['loc_og_t2t_mou_jul'] + churn_data_high['loc_ic_t2t_mou_jul']
churn_data_high['total_loc_t2t_mou_aug'] = churn_data_high['loc_og_t2t_mou_aug'] + churn_data_high['loc_ic_t2t_mou_aug']

# Calculate the mean of total local T2T minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_loc_t2t_mou'] = round(churn_data_high[['total_loc_t2t_mou_jun', 'total_loc_t2t_mou_jul', 'total_loc_t2t_mou_aug']].mean(axis=1), 2)

# Calculate the total STD T2T minutes of usage for each month
churn_data_high['total_std_t2t_mou_jun'] = churn_data_high['std_og_t2t_mou_jun'] + churn_data_high['std_ic_t2t_mou_jun']
churn_data_high['total_std_t2t_mou_jul'] = churn_data_high['std_og_t2t_mou_jul'] + churn_data_high['std_ic_t2t_mou_jul']
churn_data_high['total_std_t2t_mou_aug'] = churn_data_high['std_og_t2t_mou_aug'] + churn_data_high['std_ic_t2t_mou_aug']

# Calculate the mean of total STD T2T minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_std_t2t_mou'] = round(churn_data_high[['total_std_t2t_mou_jun', 'total_std_t2t_mou_jul', 'total_std_t2t_mou_aug']].mean(axis=1), 2)

# Calculate the total local T2T minutes of usage for each month in churn_data_unseen
churn_data_unseen['total_loc_t2t_mou_jun'] = churn_data_unseen['loc_og_t2t_mou_jun'] + churn_data_unseen['loc_ic_t2t_mou_jun']
churn_data_unseen['total_loc_t2t_mou_jul'] = churn_data_unseen['loc_og_t2t_mou_jul'] + churn_data_unseen['loc_ic_t2t_mou_jul']
churn_data_unseen['total_loc_t2t_mou_aug'] = churn_data_unseen['loc_og_t2t_mou_aug'] + churn_data_unseen['loc_ic_t2t_mou_aug']

# Calculate the mean of total local T2T minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_loc_t2t_mou'] = round(churn_data_unseen[['total_loc_t2t_mou_jun', 'total_loc_t2t_mou_jul', 'total_loc_t2t_mou_aug']].mean(axis=1), 2)

# Calculate the total STD T2T minutes of usage for each month in churn_data_unseen
churn_data_unseen['total_std_t2t_mou_jun'] = churn_data_unseen['std_og_t2t_mou_jun'] + churn_data_unseen['std_ic_t2t_mou_jun']
churn_data_unseen['total_std_t2t_mou_jul'] = churn_data_unseen['std_og_t2t_mou_jul'] + churn_data_unseen['std_ic_t2t_mou_jul']
churn_data_unseen['total_std_t2t_mou_aug'] = churn_data_unseen['std_og_t2t_mou_aug'] + churn_data_unseen['std_ic_t2t_mou_aug']

# Calculate the mean of total STD T2T minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_std_t2t_mou'] = round(churn_data_unseen[['total_std_t2t_mou_jun', 'total_std_t2t_mou_jul', 'total_std_t2t_mou_aug']].mean(axis=1), 2)

### Identify "t2m" columns

In [ ]:
# Define regex pattern to match columns containing "t2m"
pattern = r't2m'
# Create a list to hold columns containing "t2m"
t2m_cols = sorted([col for col in churn_data_high.columns if re.search(pattern, col, re.IGNORECASE)])
# Print identified columns
print('Columns with "t2m":', t2m_cols)

In [ ]:
# Calculate the total local t2m minutes of usage for each month
churn_data_high['total_loc_t2m_mou_jun'] = churn_data_high['loc_og_t2m_mou_jun'] + churn_data_high['loc_ic_t2m_mou_jun']
churn_data_high['total_loc_t2m_mou_jul'] = churn_data_high['loc_og_t2m_mou_jul'] + churn_data_high['loc_ic_t2m_mou_jul']
churn_data_high['total_loc_t2m_mou_aug'] = churn_data_high['loc_og_t2m_mou_aug'] + churn_data_high['loc_ic_t2m_mou_aug']

# Calculate the mean of total local t2m minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_loc_t2m_mou'] = round(churn_data_high[['total_loc_t2m_mou_jun', 'total_loc_t2m_mou_jul', 'total_loc_t2m_mou_aug']].mean(axis=1), 2)

# Calculate the total STD t2m minutes of usage for each month
churn_data_high['total_std_t2m_mou_jun'] = churn_data_high['std_og_t2m_mou_jun'] + churn_data_high['std_ic_t2m_mou_jun']
churn_data_high['total_std_t2m_mou_jul'] = churn_data_high['std_og_t2m_mou_jul'] + churn_data_high['std_ic_t2m_mou_jul']
churn_data_high['total_std_t2m_mou_aug'] = churn_data_high['std_og_t2m_mou_aug'] + churn_data_high['std_ic_t2m_mou_aug']

# Calculate the mean of total STD t2m minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_std_t2m_mou'] = round(churn_data_high[['total_std_t2m_mou_jun', 'total_std_t2m_mou_jul', 'total_std_t2m_mou_aug']].mean(axis=1), 2)


### Identify "t2o" columns

In [ ]:
# Define regex pattern to match columns containing "t2o"
pattern = r't2o'
# Create a list to hold columns containing "t2o"
t2o_cols = sorted([col for col in churn_data_high.columns if re.search(pattern, col, re.IGNORECASE)])
# Print identified columns
print('Columns with "t2o":', t2o_cols)

### Identify "t2f" columns

In [ ]:
# Define regex pattern to match columns containing "t2f"
pattern = r't2f'
# Create a list to hold columns containing "t2f"
t2f_cols = sorted([col for col in churn_data_high.columns if re.search(pattern, col, re.IGNORECASE)])
# Print identified columns
print('Columns with "t2f":', t2f_cols)

In [ ]:
# Calculate the total local T2F minutes of usage for each month
churn_data_high['total_loc_t2f_mou_jun'] = churn_data_high['loc_og_t2f_mou_jun'] + churn_data_high['loc_ic_t2f_mou_jun']
churn_data_high['total_loc_t2f_mou_jul'] = churn_data_high['loc_og_t2f_mou_jul'] + churn_data_high['loc_ic_t2f_mou_jul']
churn_data_high['total_loc_t2f_mou_aug'] = churn_data_high['loc_og_t2f_mou_aug'] + churn_data_high['loc_ic_t2f_mou_aug']

# Calculate the mean of total local T2F minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_loc_t2f_mou'] = round(churn_data_high[['total_loc_t2f_mou_jun', 'total_loc_t2f_mou_jul', 'total_loc_t2f_mou_aug']].mean(axis=1), 2)

# Calculate the total STD T2F minutes of usage for each month
churn_data_high['total_std_t2f_mou_jun'] = churn_data_high['std_og_t2f_mou_jun'] + churn_data_high['std_ic_t2f_mou_jun']
churn_data_high['total_std_t2f_mou_jul'] = churn_data_high['std_og_t2f_mou_jul'] + churn_data_high['std_ic_t2f_mou_jul']
churn_data_high['total_std_t2f_mou_aug'] = churn_data_high['std_og_t2f_mou_aug'] + churn_data_high['std_ic_t2f_mou_aug']

# Calculate the mean of total STD T2F minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_std_t2f_mou'] = round(churn_data_high[['total_std_t2f_mou_jun', 'total_std_t2f_mou_jul', 'total_std_t2f_mou_aug']].mean(axis=1), 2)

# Calculate the total local T2F minutes of usage for each month
churn_data_unseen['total_loc_t2f_mou_jun'] = churn_data_unseen['loc_og_t2f_mou_jun'] + churn_data_unseen['loc_ic_t2f_mou_jun']
churn_data_unseen['total_loc_t2f_mou_jul'] = churn_data_unseen['loc_og_t2f_mou_jul'] + churn_data_unseen['loc_ic_t2f_mou_jul']
churn_data_unseen['total_loc_t2f_mou_aug'] = churn_data_unseen['loc_og_t2f_mou_aug'] + churn_data_unseen['loc_ic_t2f_mou_aug']

# Calculate the mean of total local T2F minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_loc_t2f_mou'] = round(churn_data_unseen[['total_loc_t2f_mou_jun', 'total_loc_t2f_mou_jul', 'total_loc_t2f_mou_aug']].mean(axis=1), 2)

# Calculate the total STD T2F minutes of usage for each month
churn_data_unseen['total_std_t2f_mou_jun'] = churn_data_unseen['std_og_t2f_mou_jun'] + churn_data_unseen['std_ic_t2f_mou_jun']
churn_data_unseen['total_std_t2f_mou_jul'] = churn_data_unseen['std_og_t2f_mou_jul'] + churn_data_unseen['std_ic_t2f_mou_jul']
churn_data_unseen['total_std_t2f_mou_aug'] = churn_data_unseen['std_og_t2f_mou_aug'] + churn_data_unseen['std_ic_t2f_mou_aug']

# Calculate the mean of total STD T2F minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_std_t2f_mou'] = round(churn_data_unseen[['total_std_t2f_mou_jun', 'total_std_t2f_mou_jul', 'total_std_t2f_mou_aug']].mean(axis=1), 2)

### Identify "t2c" columns

In [ ]:
print(findCustomColumnTypes(churn_data,"all_search"))

In [ ]:
# Calculate the mean of total local T2C minutes of usage across June, July, and August and round to 2 decimal places
churn_data_high['mean_total_loc_t2c_mou'] = round(churn_data_high[t2c_cols].mean(axis=1), 2)
# Calculate the mean of total local T2F minutes of usage across June, July, and August and round to 2 decimal places
churn_data_unseen['mean_total_loc_t2c_mou'] = round(churn_data_high[t2c_cols].mean(axis=1), 2)

### Drop actual columns from dataset for calculation

In [ ]:
churn_data_high_copy2=churn_data_high.copy()
churn_data_unseen_copy2=churn_data_unseen.copy()

In [ ]:
# Define base regex patterns without month suffix
base_patterns = [
    r'loc_og_mou_', r'loc_ic_mou_',
    r'std_og_mou_', r'std_ic_mou_',
    r'isd_og_mou_', r'isd_ic_mou_',
    r'roam_og_mou_', r'roam_ic_mou_',
    r'arpu_2g_', r'arpu_3g_',
    r'vol_2g_mb_', r'vol_3g_mb_',
    r't2t',r't2m',r't2o',r't2f',r't2c'
]
# Create a list to hold columns to drop
columns_to_drop = []
# Combine all base patterns into a single regex pattern with | (OR)
combined_base_pattern = '|'.join(base_patterns)
# Define regex pattern to exclude columns with "total"
exclude_total_pattern = r'^(?!.*(total|average|avg|mean)).*$'
# Find columns that match any of the base patterns
for col in churn_data_high.columns :
    if re.search(combined_base_pattern, col) and re.match(exclude_total_pattern, col):
        columns_to_drop.append(col)
print(columns_to_drop)

In [ ]:
## dropping these columns
churn_data_high.drop(columns_to_drop,axis=1,inplace=True)
## dropping these columns
churn_data_unseen.drop(columns_to_drop,axis=1,inplace=True)
## resetting the index
churn_data_high.reset_index(inplace=True,drop=True)
churn_data_unseen.reset_index(inplace=True,drop=True)
# Print the shape of the DataFrame after imputation for the training set
print("Seen Set Shape :- ", churn_data_high.shape)
# Separator line for clarity
print("=" * 120)
# Print the shape of the DataFrame after imputation for the test set
print("Unseen Set Shape :- ", churn_data_unseen.shape)

### Final Check - Missing Values on High Valued Customer dataset

In [ ]:
## Lets check the object columns with missing values 
cut_off=0
missing_values_cols_seen=getMissingValues(churn_data_high,cut_off,"Seen","all types")

### Outliers treatment

In [ ]:
### Identify the numerical columns for outlier treatment
num_cols=list(churn_data_high.select_dtypes(include=['int64', 'float64']).columns)
num_cols.remove(target_col)
# Dictionary to store the percentage of 0s for each column
zero_percentages = {}
# Loop through each numeric column to calculate the percentage of 0 values
for col in num_cols:
    zero_count = (churn_data_high[col] == 0).sum()
    total_count = churn_data_high[col].shape[0]
    zero_percentage = (zero_count / total_count) * 100
    zero_percentages[col] = zero_percentage
# Filter columns where the percentage of 0s is less than 40%
filtered_num_cols = [col for col, perc in zero_percentages.items() if perc < 40]

In [ ]:
churn_data_high[[col for col in num_cols if col not in filtered_num_cols]].describe()

In [ ]:
churn_data_high=remove_outliers(churn_data_high,filtered_num_cols,0.05,0.95,1.5)

### Check Data Imbalance in Train Set if any

In [ ]:
# Check data imbalance in the target column i.e., 'churn_probability' column of the churn_data_high DataFrame
round ( 100 * churn_data_high[target_col].value_counts(normalize=True) , 2)

#### The proportions are as follows:
 - 91.75% of the customers are classified as class 0 (not likely to churn).
 - 8.25% of the customers are classified as class 1 (likely to churn).
#### Points to remember:
1. This indicates a significant class imbalance, with a much larger proportion of customers predicted not to churn.
2. Such imbalance may affect the performance of predictive models, as they might be biased towards the majority class (class 0).
3. The imbalance could potentially impact model performance. 
4. We will need to address this imbalance in subsequent steps to ensure a more balanced and effective model.
### Before scaling the data, as mentioned previously, we will handle the class imbalance

# 3. Exploratory Data Analysis:

### Update Numerical Categorical Date Columns for EDA

In [ ]:
# Identify unique values in columns
# Print the number of columns in the training set where all values are the same
constant_cols_train = churn_data_high.columns[is_column_constant_custom(churn_data_high, churn_data_high.columns, 7)]
print("Seen Set: Number of columns where all values are the same:", len(constant_cols_train))
print("Columns with constant values in Seen Set:", constant_cols_train.tolist())

In [ ]:
### Identify the numerical columns
num_cols=list(churn_data_high.select_dtypes(include=['int64', 'float64']).columns)
num_cols=sorted(list(set(num_cols) - set(constant_cols_train)))
print(num_cols)

# Separator
print("="*120)

### Identify the object columns
obj_cols=sorted(list(churn_data_high.select_dtypes(include=['object']).columns))
print(obj_cols)

# Separator
print("="*120)

### Identify the date columns
date_cols=sorted(list(churn_data_high.select_dtypes(include=['<M8[ns]']).columns))
print(date_cols)

# Separator
print("="*120)

### Identify the categorical columns
cat_cols=sorted(list( set(constant_cols_train) - (set(num_cols) | set(obj_cols) | {target_col} | set(date_cols)) ))
print(cat_cols)

### Identify Columns Starting with "total" or "avg" or "mean"

In [ ]:
# Define regex pattern to match columns starting with "total" or "avg" or "mean"
pattern = r'^total|^avg|^mean'
# Create a list to hold columns that start with "total" or "avg" or "mean"
spl_num_cols = sorted([col for col in churn_data_high.columns if re.match(pattern, col)])
# Print identified columns
print("Columns starting with 'total' or 'avg':", spl_num_cols)

### Univariate Analysis

In [ ]:
# Set the default Seaborn style and enable grid
sns.set(style='darkgrid')

In [ ]:
plt.figure(figsize=[5,2])
sns.countplot(data=churn_data_high, x=target_col, palette='deep')
plt.xlabel('Churn status', fontsize=10)
plt.ylabel('Customer count',fontsize=10)
plt.title('Customer counts Vs Churn Status')
plt.show()

### Numerical Columns

In [ ]:
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(spl_num_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 6
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
# Create subplots
for ax, col in enumerate(spl_num_cols):
    plt.subplot(fig_num_rows, fig_num_cols, ax + 1)
    # Generate a plot for the current column
    sns.distplot( churn_data_high[col] , color='#55A868' )
    # Set the title for the current subplot
    plt.title( f' Univariate Analysis of variable using Distplot - {col} ' )
# Adjust subplot parameters for a tight layout
plt.tight_layout()
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

### Categorical Columns

In [ ]:
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(cat_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 6
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
# Loop through each column to create a subplot
for ax , col in enumerate(cat_cols):
    # Create a subplot for each column
    plt.subplot(fig_num_rows, fig_num_cols, ax+ 1)
    # Order the data
    sort_order=sorted(churn_data_high[col].unique())
    # Generate a  plot for the current column
    sns.countplot( data=churn_data_high , x=col , palette='deep' ,order=sort_order)
    # Set the title for the current subplot
    plt.title( f'Univariate Analysis of Variable - {col}' )
# Adjust subplot parameters for a tight layout
plt.tight_layout()
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

### Bivariate Analysis

In [ ]:
# Numerical Vs Categorical
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(spl_num_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 6
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
# Loop through each column to create a subplot
for ax, col in enumerate(spl_num_cols):
    # Create a subplot for each column
    plt.subplot(fig_num_rows, fig_num_cols, ax+ 1)
    sns.boxplot(x=churn_data_high[target_col], y=churn_data_high[col], palette='deep')
    plt.title(f' Bivariate Analysis of Variable - {col} Vs. {target_col} ')
# Adjust layout
plt.tight_layout()
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
# Categorical Vs Categorical
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(cat_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 6
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
# Create subplots
for ax, col in enumerate(cat_cols):
    # Create a subplot for each column
    plt.subplot(fig_num_rows, fig_num_cols, ax+ 1)
    sort_order = sorted(churn_data_high[col].unique())
    sns.countplot(data=churn_data_high, x=col, hue=target_col, palette='deep', order=sort_order)
    plt.title(f' Bivariate Analysis of Variable - {col} Vs. {target_col} ')
# Adjust layout
plt.tight_layout()
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

### Multivariate Analysis

In [ ]:
#Heatmap of Specific Numerical Variables
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(cat_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 6
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
mask = np.triu(churn_data_high[spl_num_cols].corr())
sns.heatmap(churn_data_high[spl_num_cols].corr(),mask=mask,annot=True,cmap="Reds")
plt.title('Heatmap of Specific Numerical Variables')
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
#Heatmap of Specific Numerical Variables
# Start time
start_time = time.time()
# Define the number of subplots (adjust according to your needs)
num_plots = len(num_cols)
# Number of columns in the subplot grid
fig_num_cols = 4  
fig_num_rows = (num_plots // fig_num_cols) + (num_plots % fig_num_cols != 0)  # Calculate number of rows needed
# Set the figure size (width, height) in inches
fig_width = fig_num_cols * 6
fig_height = fig_num_rows * 2
# Create the figure
plt.figure(figsize=(fig_width, fig_height))
mask = np.triu(churn_data_high[num_cols].corr())
sns.heatmap(churn_data_high[num_cols].corr(),mask=mask,cmap="Reds")
plt.title('Heatmap of Numerical Variables')
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

### Analyze August Data Usage and Recharge Amounts with Various Hue Indicators"

In [ ]:
# Start time
start_time = time.time()
# Define columns for the X and Y axes for June data
cols_x_aug = 'total_month_rech_amt_aug'  # Total recharge amount in August
cols_y_aug = 'total_vol_2g_3g_mb_aug'    # Total data usage in AUgust
# Define the columns to be used as hue (color) in the scatter plots for August data
cols_hue_aug = ['churn_probability', 'fb_user_aug', 'monthly_2g_aug', 'monthly_3g_aug']
# Create scatter plots with different hues for the selected columns
for hue_col in cols_hue_aug:
    sns.pairplot(churn_data_high, x_vars=cols_x_aug, y_vars=cols_y_aug, hue=hue_col, 
                 kind='scatter', palette='deep', aspect=0.8)
    plt.title(f'Churn Probability, Social Media Usage, and 2G/3G Monthly Usage (August) - Scatter Plot with {hue_col}')
    plt.ylabel('Total Data Used in August (MB)')
    plt.xlabel('Total Amount Spent in August')
# Adjust layout
plt.tight_layout()
# Display the plots
plt.show()
# End time
end_time = time.time()
# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

### Observations from the Bivariate Analysis :
    
    - Call Usage: Most users show low call usage, with some heavy users as outliers.
    - Churn Probability: Higher usage generally correlates with a higher likelihood of churn.
    - Outliers: Numerous outliers suggest a few users have extreme usage patterns.
    - Monthly Comparison: Median usage is consistent across months, but individual behaviors vary.
    - Roaming Usage: High variance in roaming usage, with heavy roamers showing more churn risk.
    - Data Usage (2G/3G): Higher data consumption is linked to a greater chance of churn.
    - Service Type Influence: Frequent STD and ISD callers tend to have a higher churn probability.
    - Facebook and similar social networking sites user : Most users are not Facebook users across all three months, with a consistent trend of slightly higher churn among non-users.
    - Monthly 2G Usage: Across June, July, and August, the majority of users have minimal 2G usage, with a higher churn rate observed among users with increased 2G usage.
    - Monthly 3G Usage: Consistently low 3G usage across all three months, with higher 3G usage correlating with a higher churn probability.
    - Night Pack Users: Most users do not use night packs in any of these months, with a small segment of night pack users showing a higher churn rate.
    
### Observations from the Multivariate Analysis:
    - High correlation among ARPU variables in June, July, and August.
    - Strong positive correlations among total_rech_amt variables for all months.
    - Moderate correlation between outgoing and incoming minutes of usage.
    - Weak correlation between total recharge numbers and ARPU.
    - Stable user behavior across different months.
    - Potential redundancy in variables due to high correlations.
    - Consistent trends in total minutes of usage (total_mou) across months.
    - Recharge and usage patterns are closely linked.
    - Low negative correlation between some recharge and usage metrics.
    - Higher correlations in same-category features like total_arpu and total_mou

In [ ]:
churn_data_high_pca=